# Joins

## Introducing the Datasets
- Data in the real world can be spread out across multiple tables/datasets.
- A join combines two tables/`DataFrames` together based on a logical criteria.
- To **join** means to link or connect.
- In this section, we'll be exploring data from a fictional streaming service (ala Netflix).
- All CSV datasets are found within the `streaming_service` directory.

- The `movies.csv` dataset holds information on the films available on the service.

- The `plans.csv` file lists the streaming service's subscription plans.

- The `users.csv` file lists the subscribers (name, email, and the ID of their subscription plan).

- The `watch_history.csv` file is a join table that connects a user ID and a movie ID.

- The `support.csv` file is a dataset of complaints that customer support received.
- Some tickets are connected to a specific user/subscriber; other tickets are not.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#quick-reference-table
- https://docs.pola.rs/user-guide/transformations/joins/#equi-joins

## Inner Join

- An `inner` join merges rows that have a matching value in both `DataFrames`.
- Inner joins are ideal when you want to find the intersection or overlap between two datasets (the values in common).
- Polars will join rows based on the same value existing in a specified column in both tables.

<img src="images/Inner_Join.webp" alt="Inner Join Diagram" width="400" />

- Let's say I want to find every movie that every user watched.
- An inner join between `users` and `watch_history` identifies users whose ID appears in both `users.id` and `watch_history.user_id`.
- Polars excludes `null` (missing) values in matches by default.  A null is not considered equal to another null.

- Some users have watched multiple movies. Some users have watched no movies.
- An inner join will exclude users with an `id` in `users` but no `user_id` in `watch_history`. In other words, an inner join will exclude users with no watch history.
- An inner join can create multiple matches if the same user watched multiple movies. The same `user_id` may appear multiple times in the `watch_history` table and Polars will match it repeatedly with the row with the same user ID in `users`.

### The join Method
- The `join` method merges two `DataFrames` together.
- The `other` parameter sets the second `DataFrame`.
- The `how` parameter declares the join strategy.
- Polars appends a `_right` suffix to any duplicate column name from the right `DataFrame`.
- Polars exclude `watch_history.user_id` from the result. The value in that column is the same as `id`.

- Let's identify the users who watched multiple movies.
- The `is_duplicated` method returns True if a row stores a duplicate value.

- The `suffix` parameter attaches a custom suffix to the duplicate column names from the right `DataFrame`.
- The `id` column from `watch_history` becomes `id_from_watch_history`.
- The `id` column from `users` is unaffected and remains `id`.

- For inner joins, Polars excludes the matching join column from the right table (`user_id` from the `watch_history`).
- The `watch_history.user_id` column's values would match the `users.id` value.
- Pass the `coalesce` parameter an argument of `False` to include the matching columns.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#inner-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## The on Parameter
- The `on` parameter can designates the join column _if_ the column name is identical across both tables.

- Let's execute the inner join from the previous lesson.
- The `left_on` and `right_on` parameters are still valid...

- ...but the `on` parameter is cleaner.

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Full Joins
- A full join keeps merges rows from both `DataFrames`.
- Like an inner join, a full join will match rows based on a shared value across specified columns.
- Unlike an inner join, a full join will keep a row if it does not have a match in the other `DataFrame`.
- Polars will populate the remaining row values with `null` if there isn't a match with the other `DataFrame`.

<img src="images/Full_Join.webp" alt="Full Join Diagram" width="400" />

- An orphan record is an entry that doesn't have a matching record in the other table.
- A full join is ideal for identifying orphan records (users whose plan ID is invalid, plans who have no users).
- Richard Parsons has a `subscription_plan_id` (5) that doesn't exist in the `plans` `DataFrame`.
- The row entries for Richard Parsons are `null`. There are no values to pull in from a matching row in `plans` `DataFrame`.
- The Deluxe plan and its ID (4) has no user who subscribes to it.
- The Deluxe plan row has `null` values for the user `id`, `name`, `email`, and `subscription_plan_id`

- Let's identify the users who are subscribed to a non-existent subscription plan.

- Let's identify the plans who have no subscribing users.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#full-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Left and Right Joins
- A left join keeps all records from the left table and merges matching rows (where possible) from the right table.
- All left table rows will be present. Where there is no match, Polars will populate `null` in the right table columns.

<img src="images/Left_Join.png" alt="Left Join Diagram" width="400" />

- The first three support tickets have an associated user who we can match by ID in the `users` table.
- The fourth ticket has no `user_id`; the row's values for `name`, `email`, and `subscription_plan_id` will be null.
- A null `user_id` in the `users` table will also create `null` (missing) values.
- Polars will coalesce (combine together into one) the duplicate columns in a left join.

- A right join on the `users` `DataFrame` will accomplish the same result.
- The right table (`support`) is still the anchor table/source of truth. All of its rows will be kept.
- Polars will bring in the rows where the `users.id` matches `support.user_id`.
- The order of the columns will be different.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#left-join
- https://docs.pola.rs/user-guide/transformations/joins/#right-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Semi Join
- A semi join keeps only the left `DataFrame` rows that have a match in the right `DataFrame`.
- However, Polars does not concatenate the right `DataFrame`'s columns to the resulting `DataFrame`.
- A semi join is closer to a filter operation than a join.

- Business Case: Let's filter for the users who created at least one support ticket.
- Invoke `join` on `users` so the resulting `DataFrame` has the columns from the `users` table.

### Further Reading
- https://docs.pola.rs/user-guide/getting-started/#joining-dataframes
- https://docs.pola.rs/user-guide/transformations/joins/#semi-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Anti Join
- The anti join is the opposite of the semi join.
- The anti join keeps the left `DataFrame` rows that do not have a match in the right `DataFrame`.

- Business case: Identify the users who did not file a ticket/complaint.
- We want to find the rows with an `id` in `users` (left) that do not have a match in `support.user_id`.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#anti-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Cross Joins
- A cross join matches every row from the left `DataFrame` with every row from the right `DataFrame`.
- The strategy is called a Cartesian product.
- The resulting `DataFrame`'s length will be equal to the product of the two `DataFrame's` lengths.
- The `on` parameter is not required because every left row will be matched with every right row.

- A cross join will include all columns from both `DataFrames`.
- Polars will append `_right` to the right `DataFrame`'s duplicate columns.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#cartesian-product
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## Joining on Multiple Columns
- Polars can join `DataFrames` based on matching values across multiple columns.
- Let's imagine we are running a chain of stores.
- We manage the store's inventory in one system and the products in another.
- The inventory system stores the remaining units of each product per store.

- The price system stores each the price of each product per store.

- Review: An **inner join** merges rows where values exist and match in _both_ `DataFrames`.
- An inner join across multiple columns will merge rows based on matching values in both `store_id` and `product_id`.
- If the `store_id` matches but the `product_id` does not match, Polars does not join the rows.
- If the `product_id` matches but the `store_id` does not match, Polars does not join the rows.
- Pass the `on`/`left_on`/`right_on` parameters a list of column names.

- Review: A **full join** pulls in all rows from both `DataFrames`.
- Polars with join rows where `store_id` and `product_right` match.
- Poalrs will keep rows with no complementary matching values..
- ...but those rows will have `null` values in the other columns.

- Review: A **left join** keeps all rows from `inventory` and pulls in matching rows from `prices`.
- Polars will populate `null` for the `price` column if the `store_id` and `product_id` does not exist in `prices`.
- Polars will exclude rows from `prices` with no matching combination of `store_id` + `product_id`.

- Review: A **semi join** selects/filters the `inventory` rows where the `store_id` and `product_id` exist in `prices`.
- A semi join does not pull in the corresponding data from `prices`.
- Semi joins isolate the `store_id` and `product_id`s that exist in both tables.
- For example, a row with `store_id=1` and `product_id=101` exists in both tables.

- Review: An **anti join** selects/filters the `inventory` rows where `store_id` and `product_id` does not in `prices`.
- An anti join identifies which combinations of `inventory`'s `store_id` and `product_id` do not exist in `prices`.
- For example, a row with `store_id=2` and `product_id=103` exists in `inventory` but does not exist in the `prices` table.

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## The validate Parameter
- The `validate` parameter to the `join` method asserts on the uniqueness of the join keys.
- Think of `validate` as a safety check to confirm the `join` method is doing what we expect.
- The default validation is `m:m` (join keys can occur multiple times in both left and right `DataFrame`).
- `m:m` turns off off the validation check entirely. It doesn't matter if join key occurs 0 times, 1 time, or multiple times.

- The `1:m` (1 to many) validation checks that the join key(s) are unique in the left dataset.
- Each `student_id` must only exist once in the `students` `DataFrame` (that's the `1` part).
- There _can_ be multiple occurrences of the same `student_id` in `enrollments` (that's the `m` part).

- A `validate` parameter set to `1:1` asserts that join keys are unique in both left and right `DataFrames`.
- Each value in the left `DataFrame` can match at most once with a value in the right `DataFrame`.
- A `1:1` validation will fail here because the same `student_id` repeats in the `enrollments` `DataFrame`.

- The inverted `m:1` (many to 1) validation checks that the join key(s) are unique in the right dataset.
- There _can_ be multiple occurrences of the same `student_id` in `enrollments` (that's the `m` part).
- Each `student_id` must only exist once in the `students` `DataFrame` (that's the `1` part).
- The example is the same as the previous one but the left and right `DataFrames` are flipped.

### Further Reading
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join.html

## The join_asof Method I
- The `join_asof` method matches values on the _nearest_ match rather than an exact match.
- This technique is particularly helpful when working with timeseries data.
- A row's datetime value may not perfectly match with a datetime entry in another table.
- "as of" means "up to or before a given time." i.e., "The system was running as of 9am this morning"
- The `outages.csv` shows the timestamps of issues on a website.

- The `uptime_checks.csv` shows the timestamps of uptime checks.
- The website performs an uptime check every 10 minutes.
- A value of "OK" indicates no issue at that time.

- To use `join_asof`, sort both `DataFrames` using the `on` column (the columns whose values will be used for the join)
- A backward search selects the last row in the right `DataFrame` who key is less than or equal to the left's key.
- The default strategy is `backward`.

- A forward search selects the first row in the right `DataFrame` who key is greater than or equal to the left's key.

- A `strategy` of `nearest` finds the closest match (least distance to travel).
- If a time falls right in the middle, it will be rounded up (much like 0.5 is rounded to 1).
- Notice that Polars matches `2026-01-01 00:47:00` with `2026-01-01 00:50:00` timestamp (forward).
- Vice versa, Polars matches `2026-01-01 01:13:00` with `2026-01-01 01:10:00` timestamp (backward).

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#asof-join
- https://docs.pola.rs/api/python/stable/reference/dataframe/api/polars.DataFrame.join_asof.html

## The join_asof Method II: Tolerance
- The `tolerance` parameter sets the constraint/boundary/duration by which the match can occur in the given search direction.
- Each unit of time has a corresponding symbol:
    - `ms` for millisecond
    - `m` for minute
    - `h` for hour
    - `d` for day
    - `w` for week
    - `mo` for month
    - `q` for quarter
    - `y` for year

- For example, `5m` (5 minutes) and a `backward` strategy instructs Polars to look for a matching timestamp in the previous 5 minutes.
- The outage at `2026-01-01 00:05:00` no longer matches `2026-01-01 00:00:00` because it it outside the 4-minute window.
- Polars supplies `null` where it cannot join a complementary row.

- A unit of time can combine symbols: `6d7h5m` declares a tolerance range of "6 days, 7 hours, 5 minutes".
- There is no tolerance by default, so Polars will proceed backwards until it finds a match (if one exists).

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#asof-join

## The join_asof Method III: The by Parameter
- Some datasets will require a join by exact keys before performing an approximate match.
- The `by` parameter sets the column(s) to match on _exactly_ between the two `DataFrames`.
- The `on` parameter sets the column(s) to match on nearness between the two `DataFrames`.
- Sort the `by` columns first, then sort the `on` key for each value within them.

- Let's say we join based the on nearest timestamps.
- Here's an example where the `join_asof` will not work as expected.
- Polars correctly locates the closest timestamp...
- ...but it completely ignores the `currency` columns whose values need to match.
- We need to join on a matching currency, `_then_` find the closest timestamp.

- Let's re-import the data. We want to sort on the join column (`currency`) first, then the near-join column (`timestamp`) after.

- Polars will warn that it cannot verify the sorted nature of columns when we use `by`.
- The warning does not mean the work is incorrect.
- The first timestamp (`2025-01-01 00:13:25` for EUR) matches `2025-01-02 00:00:00` for EUR and brings over its 1.0921 value.

- Finally, let's calculate the amount paid in USD by multiplying the amount by the conversion rate.

### Further Reading
- https://docs.pola.rs/user-guide/transformations/joins/#asof-join